# ⚡ Hızlı Torrent İndirici


In [ ]:
!pip install --upgrade pip setuptools wheel -q
!pip install libtorrent==2.0.11 -q
print("✅ Kurulum tamam")


In [ ]:
import hashlib
import time
import warnings
from pathlib import Path

import ipywidgets as widgets
import libtorrent as lt
from IPython.display import clear_output, display
from google.colab import drive


drive.mount('/content/drive', force_remount=False)

DOWNLOAD_DIR = Path('/content/drive/MyDrive/Torrent')
RESUME_DIR = Path('/content/drive/MyDrive/.torrent_resume')
TMP_TORRENT_DIR = Path('/content/.tmp_torrents')

for p in (DOWNLOAD_DIR, RESUME_DIR, TMP_TORRENT_DIR):
    p.mkdir(parents=True, exist_ok=True)

session = lt.session()

with warnings.catch_warnings():
    warnings.simplefilter('ignore', DeprecationWarning)
    if hasattr(session, 'start_dht'):
        session.start_dht()
    if hasattr(session, 'start_upnp'):
        session.start_upnp()

FAST_SETTINGS = {
    'connections_limit': 2000,
    'connection_speed': 200,
    'active_downloads': 60,
    'active_seeds': 60,
    'active_limit': 200,
    'active_checking': 20,
    'request_timeout': 12,
    'peer_timeout': 30,
    'tick_interval': 200,
    'max_out_request_queue': 1500,
    'max_allowed_in_request_queue': 2000,
    'aio_threads': 8,
    'dl_rate_limit': 0,
    'ul_rate_limit': 0,
}


def apply_supported_settings(sess, settings):
    applied = []
    skipped = []

    for key, value in settings.items():
        try:
            sess.apply_settings({key: value})
            applied.append(key)
        except Exception:
            skipped.append(key)

    return applied, skipped


applied_settings, skipped_settings = apply_supported_settings(session, FAST_SETTINGS)

print(f"✅ İndirme klasörü: {DOWNLOAD_DIR}")
print(f"✅ Resume klasörü: {RESUME_DIR}")
print(f"✅ Uygulanan ayar: {len(applied_settings)}")
if skipped_settings:
    print(f"ℹ️ Atlanan (sürüm uyumsuz) ayarlar: {', '.join(skipped_settings)}")



In [ ]:
queue = []
active = []
max_concurrent = 20
checkpoint_seconds = 5


def torrent_key(kind, data):
    raw = data if isinstance(data, str) else str(data)
    return hashlib.sha1(f"{kind}:{raw}".encode()).hexdigest()


def resume_path(key):
    return RESUME_DIR / f"{key}.resume"


def load_resume(key):
    path = resume_path(key)
    return path.read_bytes() if path.exists() else None


def save_resume(handle, key):
    if not handle.is_valid():
        return
    data = lt.write_resume_data(handle)
    resume_path(key).write_bytes(data)


def tune_handle(handle):
    handle.set_max_connections(1200)
    handle.set_max_uploads(800)
    handle.set_download_limit(0)
    handle.set_upload_limit(0)


magnet_input = widgets.Textarea(
    description='Magnet',
    placeholder='magnet:?xt=urn:btih:...',
    layout=widgets.Layout(width='100%', height='90px')
)
upload = widgets.FileUpload(accept='.torrent', multiple=True)
add_magnet_btn = widgets.Button(description='Magnet Ekle', button_style='primary')
add_torrent_btn = widgets.Button(description='Torrent Ekle', button_style='primary')
start_btn = widgets.Button(description='İndirmeyi Başlat', button_style='success')
queue_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))
status_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))


def render_queue():
    with queue_out:
        clear_output(wait=True)
        print(f"Kuyruk: {len(queue)}")
        for i, item in enumerate(queue[-12:], 1):
            print(f"{i:02d}. {item['name']}")


def add_magnet(_):
    magnet = magnet_input.value.strip()
    if not magnet.startswith('magnet:?'):
        with queue_out:
            print('⚠️ Geçerli magnet girin.')
        return
    key = torrent_key('magnet', magnet)
    queue.append({'type': 'magnet', 'data': magnet, 'key': key, 'name': magnet[:80]})
    magnet_input.value = ''
    render_queue()


def add_torrents(_):
    if not upload.value:
        with queue_out:
            print('⚠️ Önce .torrent dosyası yükleyin.')
        return

    for filename, meta in upload.value.items():
        save_file = TMP_TORRENT_DIR / filename
        save_file.write_bytes(meta['content'])
        key = torrent_key('torrent', filename + str(len(meta['content'])))
        queue.append({'type': 'torrent', 'data': str(save_file), 'key': key, 'name': filename})

    upload.value.clear()
    upload._counter = 0
    render_queue()


def run_download(_):
    if not queue and not active:
        with status_out:
            clear_output(wait=True)
            print('⚠️ Kuyruk boş.')
        return

    start_btn.disabled = True
    started = time.time()
    last_checkpoint = time.time()

    while queue or active:
        while queue and len(active) < max_concurrent:
            item = queue.pop(0)
            params = lt.add_torrent_params()
            params.save_path = str(DOWNLOAD_DIR)

            if item['type'] == 'magnet':
                params.url = item['data']
            else:
                params.ti = lt.torrent_info(item['data'])

            resume_data = load_resume(item['key'])
            if resume_data:
                params.resume_data = resume_data

            handle = session.add_torrent(params)
            tune_handle(handle)
            active.append((handle, item))

        now = time.time()
        if now - last_checkpoint >= checkpoint_seconds:
            for handle, item in active:
                if handle.is_valid() and not handle.is_seed():
                    save_resume(handle, item['key'])
            last_checkpoint = now

        total_speed = 0
        total_done = 0
        lines = []

        for handle, item in active[:]:
            if not handle.is_valid():
                active.remove((handle, item))
                continue

            st = handle.status()
            if handle.is_seed():
                save_resume(handle, item['key'])
                session.remove_torrent(handle)
                active.remove((handle, item))
                continue

            pct = st.progress * 100
            speed = st.download_rate
            total_speed += speed
            total_done += st.total_done
            lines.append(f"{pct:5.1f}% | {speed/1024/1024:6.2f} MB/s | {item['name'][:55]}")

        with status_out:
            clear_output(wait=True)
            elapsed = int(time.time() - started)
            print(f"Süre: {elapsed//3600:02d}:{(elapsed%3600)//60:02d}:{elapsed%60:02d}")
            print(f"Aktif: {len(active)} | Kuyruk: {len(queue)}")
            print(f"Toplam hız: {total_speed/1024/1024:.2f} MB/s")
            print(f"İndirilen: {total_done/1024/1024/1024:.2f} GB")
            print('-' * 72)
            for row in lines[:20]:
                print(row)

        time.sleep(1)

    with status_out:
        clear_output(wait=True)
        print('✅ Tüm indirmeler tamamlandı.')

    start_btn.disabled = False


add_magnet_btn.on_click(add_magnet)
add_torrent_btn.on_click(add_torrents)
start_btn.on_click(run_download)

panel = widgets.VBox([
    magnet_input,
    widgets.HBox([add_magnet_btn, upload, add_torrent_btn, start_btn]),
    queue_out,
    status_out,
])

display(panel)
render_queue()
